In [3]:
import pandas as pd
import altair as alt
state_to_fips = {
    "Alabama": 1, "Alaska": 2, "Arizona": 4, "Arkansas": 5,
    "California": 6, "Colorado": 8, "Connecticut": 9, "Delaware": 10,
    "Florida": 12, "Georgia": 13, "Hawaii": 15, "Idaho": 16,
    "Illinois": 17, "Indiana": 18, "Iowa": 19, "Kansas": 20,
    "Kentucky": 21, "Louisiana": 22, "Maine": 23, "Maryland": 24,
    "Massachusetts": 25, "Michigan": 26, "Minnesota": 27, "Mississippi": 28,
    "Missouri": 29, "Montana": 30, "Nebraska": 31, "Nevada": 32,
    "New Hampshire": 33, "New Jersey": 34, "New Mexico": 35, "New York": 36,
    "North Carolina": 37, "North Dakota": 38, "Ohio": 39, "Oklahoma": 40,
    "Oregon": 41, "Pennsylvania": 42, "Rhode Island": 44, "South Carolina": 45,
    "South Dakota": 46, "Tennessee": 47, "Texas": 48, "Utah": 49,
    "Vermont": 50, "Virginia": 51, "Washington": 53, "West Virginia": 54,
    "Wisconsin": 55, "Wyoming": 56
}

url = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv"
df = pd.read_csv(url)

state_counts = (
    df.groupby('state', as_index=False)
    .size()
    .rename(columns={'size': 'count'})
)
state_counts['state_id'] = state_counts['state'].map(state_to_fips)

state_counts.to_csv("bfro_reports_processed.csv", index=False)

In [5]:
state_to_code = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR",
    "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE",
    "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS",
    "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD",
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS",
    "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV",
    "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY",
    "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK",
    "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT",
    "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV",
    "Wisconsin": "WI", "Wyoming": "WY"
}

url = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv"
df = pd.read_csv(url)

df["state_code"] = df["state"].map(state_to_code)

df.to_csv("bfro_reports_processed.csv", index=False)

In [3]:
# %% [python]
# HW5.1 Complete Solution - Business Licenses Analysis
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

# Load data and inspect columns
url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/licenses_fall2022.csv"
licenses = pd.read_csv(url)

# ---------------------------------------------------------------------------
# DATA CLEANING
# ---------------------------------------------------------------------------
# Find potential geographic columns (modify this list based on actual columns)
geo_cols = []
possible_geo = ['latitude', 'lat', 'longitude', 'lon', 'xcoord', 'ycoord',
                'x', 'y', 'coord', 'coordinates']
for col in possible_geo:
    if col in licenses.columns:
        geo_cols.append(col)
        
# Find date column
date_col = next((col for col in ['issue_date', 'date', 'application_date'] 
                if col in licenses.columns), None)

# Find description column
desc_col = next((col for col in ['license_description', 'description', 'type'] 
                if col in licenses.columns), None)

# ---------------------------------------------------------------------------
# VISUALIZATION 1: LICENSE TYPE DISTRIBUTION (always works)
# ---------------------------------------------------------------------------
if desc_col:
    type_dist = alt.Chart(licenses).mark_bar().encode(
        x=alt.X(f'{desc_col}:N', title='License Type', sort='-y'),
        y='count():Q',
        color=alt.value('steelblue')
    ).properties(
        width=800,
        title='Distribution of License Types'
    ).interactive()
    
    type_dist.save('license_types.json')

# ---------------------------------------------------------------------------
# VISUALIZATION 2: GEOGRAPHIC OR TEMPORAL (conditional)
# ---------------------------------------------------------------------------
chart2 = None

# Case 1: Geographic visualization possible
if len(geo_cols) >= 2 and desc_col:
    valid_geo = licenses.dropna(subset=geo_cols[:2])
    chart2 = alt.Chart(valid_geo).mark_circle().encode(
        longitude=f'{geo_cols[1]}:Q',
        latitude=f'{geo_cols[0]}:Q',
        color=alt.Color(f'{desc_col}:N', legend=None),
        tooltip=[f'{desc_col}:N', 'legal_name:N']
    ).add_params(
        alt.selection_point(
            fields=[desc_col],
            bind=alt.binding_select(
                options=valid_geo[desc_col].unique().tolist(),
                name='Select License Type:'
            )
        )
    ).transform_filter(
        f'datum.{desc_col} == param_1.{desc_col}'
    ).properties(
        width=800,
        title='License Locations' if len(geo_cols)>=2 else 'License Analysis'
    )

# Case 2: Temporal visualization
elif date_col:
    licenses[date_col] = pd.to_datetime(licenses[date_col], errors='coerce')
    valid_dates = licenses.dropna(subset=[date_col])
    chart2 = alt.Chart(valid_dates).mark_bar().encode(
        x=alt.X(f'year({date_col}):O', title='Year'),
        y='count():Q',
        color=alt.value('steelblue')
    ).properties(
        width=800,
        title='Licenses Issued Per Year'
    )

# Case 3: Fallback to simple value counts
else:
    value_counts = licenses.iloc[:, 0].value_counts().reset_index()
    chart2 = alt.Chart(value_counts).mark_bar().encode(
        x='count:Q',
        y=alt.Y('index:N', sort='-x'),
        color=alt.value('steelblue')
    ).properties(
        width=800,
        title='Value Counts of First Column'
    )

if chart2:
    chart2.save('license_secondary.json')

print("Visualizations generated successfully!")

Visualizations generated successfully!
